# Large Language Models

We're using the Ollama software in our examples. It runs open source LLMs and can be freely downloaded from https://ollama.ai/ 

Currently (8/31/2023) Ollama only supports Mac OS, with Windows and Linux coming soon. If you are on another operating system, Langchain supports Llama.cpp and other local LLM providers, as well as many commercial providers such as OpenAI's ChatGPT. See [the docs](https://python.langchain.com/docs/integrations/providers/llamacpp) for more info.

In [1]:
from langchain.chat_models import ChatOllama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import HumanMessage
# Instantiate Ollama with the wizard-vicuna model. 
# You'll need to run "ollama pull llama2:13b" to fetch this model.
# If you have < 32GB of RAM and/or have problems, try the "7b" model.
chat_model = ChatOllama(model="llama2:13b", 
                        callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [2]:
messages = [
    HumanMessage(content="Testing 1 2 3")
]

In [3]:
messages.append(chat_model(messages))

 Hello! It looks like you're testing your microphone. Is there something specific you would like to say or ask? I'm here to help with any questions you may have.

In [4]:
messages.append(HumanMessage(content='Can you explain what an LLM is?'))
messages.append(chat_model(messages))

 Sure, I'd be happy to help! An LLM, or Master of Laws, is a postgraduate degree that focuses on advanced legal studies. It is typically pursued by lawyers who wish to specialize in a particular area of law or gain additional knowledge and skills to enhance their careers.

An LLM program typically lasts for one year and consists of coursework, research, and sometimes, clinical work. Students can choose from a variety of specializations such as corporate law, intellectual property law, international law, and many others. The program is designed to provide students with a deeper understanding of legal principles, theories, and practices, as well as the skills necessary to apply their knowledge in real-world settings.

Earning an LLM degree can be beneficial for several reasons:

1. Specialization: An LLM allows lawyers to specialize in a particular area of law, which can enhance their career prospects and increase their earning potential.
2. Networking: LLM programs provide opportunities

In [5]:
messages.append(HumanMessage(content='Actually, I was talking about a "Large Language Model".'))
messages.append(chat_model(messages))

 Ah, I see! A Large Language Model (LLM) is a type of artificial intelligence model that is trained on large amounts of text data to generate language outputs that are coherent and natural-sounding. These models have become increasingly popular in recent years due to their ability to generate text that is often indistinguishable from human-written content.

LLMs can be used for a variety of applications, such as:

1. Language Translation: LLMs can be trained on large amounts of text data in multiple languages, allowing them to generate translations that are more accurate and natural-sounding than those produced by traditional machine translation systems.
2. Text Summarization: LLMs can be used to summarize long documents, such as news articles or scientific papers, into shorter, more digestible versions.
3. Chatbots and Conversational AI: LLMs can be used to power chatbots and other conversational AI systems, allowing them to understand and respond to user input in a more natural and h

----

# Code Generation

Let's try using it to generate some code

In [6]:
messages = []
messages.append(HumanMessage(content='Can you write me a Django model class for an image? Only return the code, nothing else.'))
messages.append(chat_model(messages))

 Sure! Here is a Django model class for an image:
```
class Image(models.Model):
    title = models.CharField(max_length=255)
    image = models.FileField(upload_to='images')
```

In [7]:
messages.append(HumanMessage(content='Great! Can you write a management command that accepts a folder path and creates an Image object for each jpeg file?'))
messages.append(chat_model(messages))

 Sure! Here is a Django management command that accepts a folder path and creates an Image object for each jpeg file:
```
class ImportImages(management.BaseCommand):
    help = 'Imports images from a folder'

    def handle(self, *args, **options):
        if not args:
            self.stdout.write(self.style.WARNING('Please provide a folder path as an argument'))
            return

        folder_path = args[0]

        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg') or filename.endswith('.jpeg'):
                image_path = os.path.join(folder_path, filename)
                title = os.path.basename(filename)
                image = Image(title=title, image=image_path)
                image.save()
```
This command assumes that the images are stored in a folder with the same name as the images (e.g. "image1.jpg", "image2.jpg", etc.). It uses the `os` module to list the files in the folder and then checks if each file ends with either ".jpg" or ".jpe

Not bad, but note that this code cannot be run as it is.  It uses the `Image` model it just created, but it doesn't actually import the model. So some supervision is required, even for simple code like this.

----
# Knowledge Cutoff

Each LLM's dataset necessarily ends at a certain date, and they cannot answer questions about anything that happened after that.

In [8]:
chat_model([
    HumanMessage(content='When was LLaMA released?')
])

 I'm just an AI and do not have a specific release date as I am a continuous work in progress. I am an ongoing project of Meta AI, and my training and development are ongoing to improve my language understanding and generation capabilities. My predecessor, the chatbot "LLaMA," was released in 2018, but I have been developed and improved upon since then.

AIMessage(content=' I\'m just an AI and do not have a specific release date as I am a continuous work in progress. I am an ongoing project of Meta AI, and my training and development are ongoing to improve my language understanding and generation capabilities. My predecessor, the chatbot "LLaMA," was released in 2018, but I have been developed and improved upon since then.', additional_kwargs={}, example=False)

So even though we are using LLaMA, it has no current knowledge of itself, at least inherently.

This is a solvable problem, and one of the problems that the Retrieval Augmented Generation (RAG) architecture is meant to solve.

----

# Hallucinations

These models will just make stuff up. They are not capable of discerning reality from fiction in a lot of cases.

In [9]:
chat_model([
    HumanMessage(content='Tell me the history of the first colony on Mars.')
])

 The history of the first human colony on Mars is a story that spans decades and involves many players, from visionary scientists to daring pioneers. Here's a brief overview:

1. Early Exploration (2020s-2030s): In the early 21st century, NASA and other space agencies around the world began sending robotic missions to Mars to explore the planet and gather data. These missions discovered evidence of water ice and potential biosignatures, which fueled interest in human exploration.
2. Establishing a Base (2040s): As technology advanced and the cost of space travel decreased, private companies like SpaceX and Blue Origin began developing reusable rockets capable of transporting humans to Mars. In the 2040s, NASA and these companies collaborated to establish the first human base on Mars, which was dubbed "New Eden."
3. Early Challenges (2050s): The early years of New Eden were marked by challenges such as harsh weather conditions, limited resources, and isolation from Earth. However, the p

AIMessage(content=' The history of the first human colony on Mars is a story that spans decades and involves many players, from visionary scientists to daring pioneers. Here\'s a brief overview:\n\n1. Early Exploration (2020s-2030s): In the early 21st century, NASA and other space agencies around the world began sending robotic missions to Mars to explore the planet and gather data. These missions discovered evidence of water ice and potential biosignatures, which fueled interest in human exploration.\n2. Establishing a Base (2040s): As technology advanced and the cost of space travel decreased, private companies like SpaceX and Blue Origin began developing reusable rockets capable of transporting humans to Mars. In the 2040s, NASA and these companies collaborated to establish the first human base on Mars, which was dubbed "New Eden."\n3. Early Challenges (2050s): The early years of New Eden were marked by challenges such as harsh weather conditions, limited resources, and isolation fr

----

Cool story, but it hasn't happened. LLMs will confidently give you made up information. This can also be combatted with the RAG architecture, and potentially with system prompts as covered in the Prompting notebook.